In [ ]:
import sys
sys.path.insert(1, '/home/mlfm/tam/python_graph_utilities_v2/Codes/')
import torch
from dataset import OneDDatasetBuilder, OneDDatasetLoader

# dataset = OneDDatasetBuilder(
#     raw_dir='/data1/tam/datasets',
#     root_dir='/data1/tam/downloaded_datasets_new_test',
#     data_names=['CEMENT011'],
#     time_names=[str(i).zfill(3) for i in range(201)],
#     data_type=torch.float32
# )

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='normalized',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)
# dataset[0]

batched_dataset = dataset.batching(
    batch_size = None,
    batch_n_times = 2, 
    recursive = True, 
    sub_dir='/two_timesteps', 
    step=1
)

In [ ]:
batched_dataset[0]

In [ ]:
time = torch.zeros(dataset[0].pressure.size())
timestep = 4.8 / 200
for i in range(time.size(1)):
    time[:,i] = i * timestep
for i in range(time.size(1)):
    print(time[:,i])

In [ ]:
batched_dataset = dataset.batching(
    batch_size = 100,
    batch_n_times = 50, 
    recursive = True, 
    sub_dir='/normed_and_batched_100_50', 
    step=1
)

In [ ]:
batched_dataset[1000]

In [ ]:
import matplotlib.pyplot as plt

normed_dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='normalized',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)

data = normed_dataset[0]
print(data)

fig, ax = plt.subplots(3,2)
fig.set_figheight(15)
fig.set_figwidth(10)

ax[0,0].hist(data.node_attr[:, 0], bins=100)
ax[0,0].set_title('coordinate_x')
ax[0,1].hist(data.node_attr[:, 1], bins=100)
ax[0,1].set_title('coordinate_y')

ax[1,0].hist(data.edge_attr[:, 0], bins=100)
ax[1,0].set_title('length')
ax[1,1].hist(data.edge_attr[:, 5], bins=100)
ax[1,1].set_title('volume_1')

ax[2,0].hist(data.pressure.flatten(), bins=100)
ax[2,0].set_title('pressure')
ax[2,1].hist(data.flowrate.flatten(), bins=100)
ax[2,1].set_title('flowrate')

In [ ]:
y = dataset[0].edge_attr

i=1
print(y[:,i])

import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import PowerTransformer, MinMaxScaler, RobustScaler, QuantileTransformer, StandardScaler

transform1 = PowerTransformer()
transform1.fit(y)
transform1.lambdas_=[-0.24738493, -3.92501757,  1.07366881,  0.31679522,  1.95672088, -0.19925361, -0.16171953,  0.75660708]
# transform1.lambdas_[1] = -1.
y1 = transform1.transform(y)

plt.xlim([-20,20])
plt.hist(y[:,i], bins=100)
plt.show()

# plt.xlim([-20,20])
plt.hist(y1[:,i], bins=10)
plt.show()
plt.plot(y1[:,i])
plt.show()

# plt.xlim([-20,20])
# plt.hist(y2, bins=50)
# plt.show()

In [ ]:
var = []
for i in range(dataset.len()):
    data = dataset[i].edge_attr
    var.append(data)
var = torch.cat(var, dim=0)

from sklearn.preprocessing import PowerTransformer
transform = PowerTransformer()
transform.fit(var)
print(transform.lambdas_)
_var = transform.transform(var)


# from sklearn.preprocessing import MinMaxScaler
# transform = MinMaxScaler()
# transform.fit(var)
# _var = transform.transform(var)
# _var.min(), _var.max()

var0 = dataset[0].edge_attr
_var0 = transform.transform(var0)

In [ ]:
print(_var.mean(axis=0), _var.var(axis=0), _var.min(axis=0), _var.max(axis=0))
import matplotlib.pyplot as plt
y = _var[:,1]
plt.hist(y, bins=100)
plt.show()

print(_var0.mean(axis=0), _var0.var(axis=0), _var0.min(axis=0), _var0.max(axis=0))
y0 = _var0[:,1]
plt.hist(y0, bins=100)
plt.show()

In [ ]:
import pickle
from sklearn.preprocessing import MinMaxScaler

x = dataset[0].node_attr
scaler = MinMaxScaler()
scaler.fit(x)

with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)

In [ ]:
import torch
import numpy as np
from torch.autograd import Variable

y = dataset[0].flowrate


    
f = torch.gradient(y, spacing=4.8/200, dim=1)[0]
f = torch.clamp(f, min=-1, max=1)

import matplotlib.pyplot as plt
# plt.ylim([-10,10])
plt.plot(y[100])
plt.show()

# plt.ylim([-10,10])
plt.plot(f[100])
plt.show()
f.size(), y.size()

In [ ]:
# load data
import sys
sys.path.insert(1, '/home/mlfm/tam/python_graph_utilities_v2/Codes/')
import torch
from dataset import OneDDatasetBuilder, OneDDatasetLoader

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='normalized',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)
data = dataset[0]

# load and test model
from networks import MeshGraphNet

model = MeshGraphNet(
    node_in = (data.node_attr.size(1), True),
    node_out = (2, True), 
    edge_in = (data.edge_attr.size(1), True),
    edge_out = (0, False),
    hidden_size = 128,
    n_hiddens = 10,
    aggregation = 'sum'
)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5, weight_decay=1e-3)

x = dataset[0].node_attr.float()
edge_index = dataset[0].edge_index
edge_attr = dataset[0].edge_attr.float()

print(x.size(), edge_attr.size())

# out_node, out_edge = model.forward(x=x, edge_index=edge_index, edge_attr=edge_attr)

# print(out_node.size())

# calculate derivative
T = 4.8
nt = data.number_of_timesteps
pressure_grad = torch.gradient(data.pressure, spacing=T / (nt-1), dim=1)[0]
flowrate_grad = torch.gradient(data.flowrate, spacing=T / (nt-1), dim=1)[0]
print(pressure_grad.size(), flowrate_grad.size())
F_grad_timestep = torch.cat([pressure_grad[:,50].unsqueeze(1), flowrate_grad[:,50].unsqueeze(1)], dim=1)
# F_grad_timestep = torch.clamp(F_grad_timestep, min=-1, max=1)
F_grad_timestep = F_grad_timestep.float()
print(F_grad_timestep.size())
F_timestep_prev = torch.cat([data.pressure[:,49].unsqueeze(1), data.flowrate[:,49].unsqueeze(1)], dim=1)
F_timestep = torch.cat([data.pressure[:,50].unsqueeze(1), data.flowrate[:,50].unsqueeze(1)], dim=1)
print(F_timestep.size())

# loss
mse = torch.nn.MSELoss()

for i in range(500):
    out_node, out_edge = model.forward(x=x, edge_index=edge_index, edge_attr=edge_attr)
    grad_loss = mse(F_grad_timestep, out_node)

    out_F = F_timestep_prev + out_node * (T / (nt-1))
    value_loss = mse(F_timestep, out_F)

    loss = value_loss
    loss.backward()
    optimizer.step()
    print(loss.item())

In [ ]:
import matplotlib.pyplot as plt

field = 0

y1 = F_grad_timestep[:,field]
y2 = out_node[:,field].detach()
plt.plot(y1)
plt.plot(y2)
plt.show()

plt.hist(y1, bins=50)
plt.hist(y2, bins=50)
plt.show()

y3 = F_timestep[:,field]
y4 = out_F[:,field].detach()
plt.plot(y3)
plt.plot(y4)
plt.show()

plt.hist(y3, bins=50)
plt.hist(y4, bins=50)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
ending_node_id = 40000

current_node_id = ending_node_id
path = []
while True:
    path.append(current_node_id)
    current_edge_ids = np.where(edge_index[1] == current_node_id)[0]
    if len(current_edge_ids) < 1:
        break
    current_edge_id = current_edge_ids[0]
    current_node_id = edge_index[0][current_edge_id]

path = list(reversed(path))
plt.plot([y3[i] for i in path])
plt.plot([y4[i] for i in path])
plt.show()

In [ ]:
import sys
sys.path.insert(1, '/home/mlfm/tam/python_graph_utilities_v2/Codes/')
import torch
from networks import PARC
from dataset import OneDDatasetBuilder, OneDDatasetLoader

model = PARC(
    n_fields=2,
    n_timesteps=201,
    n_meshfields=(3,8), # Tuple(n_node_fields, n_mesh_fields)
    n_bcfields=0,
    n_hiddenfields=32,
    n_hiddens=10
)

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='normalized',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)
data = dataset[0]
mesh_features = (data.node_attr.float(), data.edge_attr.float())
edge_index = data.edge_index
F_initial = torch.cat([data.pressure[:,0].unsqueeze(1), data.flowrate[:,0].unsqueeze(1)], dim=-1).float()

out, out_dot = model.forward(
    F_initial = F_initial,
    mesh_features = mesh_features,
    edge_index = edge_index,
    F_boundary = None,
    timesteps = 4.8 / 200
)
out.size()

In [ ]:
import sys
sys.path.insert(1, '/home/mlfm/tam/python_graph_utilities_v2/Codes/')
import torch
from dataset import OneDDatasetBuilder, OneDDatasetLoader

# dataset = OneDDatasetBuilder(
#     raw_dir='/data1/tam/datasets',
#     root_dir='/data1/tam/downloaded_datasets_new_test',
#     data_names=['CEMENT011'],
#     time_names=[str(i).zfill(3) for i in range(201)],
#     data_type=torch.float32
# )

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='processed',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)

from networks_lstm import MessageNet, AggregateNet
msg_net = MessageNet()
agg_net = AggregateNet()

data = dataset[0]
edge_attr = agg_net(data.edge_attr, data.edge_index)
# is_terminal = edge_attr[:, 4] == 4
pressure = data.pressure #msg_net(data.pressure, data.edge_index)
print(edge_attr.size(), pressure.size())
# boundary_pressure = torch.transpose(torch.mul(torch.transpose(pressure,0,1), is_terminal), 0,1)
# print(boundary_pressure)

In [ ]:
# load data
import sys
sys.path.insert(1, '/data1/tam/python_graph_utilities_v2/Codes/')
import torch
from dataset import OneDDatasetBuilder, OneDDatasetLoader

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='normed_and_batched',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)
data = dataset[0]

from networks_v2 import MeshGraphNet
model = MeshGraphNet(
    node_input_size=6,
    node_output_size=2,
    edge_input_size=8,
    edge_output_size=0,
    n_hiddens=5,
    hidden_size=128,
    n_latents=5,
    latent_size=10
)

# x = torch.cat([data.pressure[:,0].unsqueeze(1), data.flowrate[:,0].unsqueeze(1)], dim=1).float()
# edge_index = data.edge_index
# edge_attr = data.edge_attr.float()

# x = torch.cat([x, data.node_attr], dim=1).float()

# time = torch.zeros(dataset[0].pressure.size())
# timestep = 4.8 / 200
# for i in range(time.size(1)):
#     time[:,i] = i * timestep

# x = torch.cat([x, time[:,10].unsqueeze(1)], dim=1).float()

# model(x, edge_index, edge_attr)

from networks_v2 import recurrent_formulation

output = recurrent_formulation(model, data, forward_sequence=True)
output_true = torch.cat([data.pressure.unsqueeze(-1), data.flowrate.unsqueeze(-1)], dim=-1)

print(output.size(), output_true.size())

criterion = torch.nn.MSELoss()
loss = criterion(output_true, output)

loss

In [ ]:
import sys
import torch.nn as nn
sys.path.insert(1, '/data1/tam/python_graph_utilities_v2/Codes/')
from networks_v1 import MeshGraphNet, build_mlp

# model = MeshGraphNet(
#     node_input_size=(3,True),
#     node_output_size=(2,True),
#     edge_input_size=(8,True),
#     edge_output_size=(1, False),
#     n_hidden=5,
#     hidden_size=128,
#     n_latent=3,
#     latent_size=128
# )
n_field = 2
latent_size = 128
n_boundaryfield = 0
n_hidden = 4
hidden_size = 128
n_latent = 3
latent_size = 128

model = MeshGraphNet(
    node_input_size=(n_field+latent_size+n_boundaryfield+1, True), # time + 1
    edge_input_size=(latent_size, True),
    node_output_size=(n_field, True),
    edge_output_size=(1, False),
    n_hidden=n_hidden, # decoder depth
    hidden_size=hidden_size,
    n_latent=n_latent, # processor depth
    latent_size=latent_size,
    n_hidden_per_processor=2
)

for i in range(len(model.processor)):
    print(i, model.processor[i].edge_fn, model.processor[i].node_fn)

In [ ]:
import sys
sys.path.insert(1, '/data1/tam/python_graph_utilities_v2/Codes/')
import torch
from dataset import OneDDatasetBuilder, OneDDatasetLoader
from plot import *

# Load dataset
dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new1',
    sub_dir='processed',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)

dataset = dataset.normalizing(
    sub_dir='normalized',
    scalers = {
        'node_attr' : ['minmax_scaler', 0],
        'edge_attr' : ['quantile_transformer', 0],
        'pressure' : ['quantile_transformer', None],
        'flowrate' : ['quantile_transformer', None]
    }
)

In [ ]:
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, '/home/mlfm/tam/python_graph_utilities_v2/Codes/')
import torch
from dataset import OneDDatasetBuilder, OneDDatasetLoader, node_to_edge, edge_to_node

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_WT',
    sub_dir='normalized_batched',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)

data = dataset[0]
x = data.node_attr[:,0]
y = data.node_attr[:,1]
z = data.node_attr[:,2]
length = data.edge_attr[:,0]
diam = data.edge_attr[:,1]
gen = data.edge_attr[:,2]
lobe = data.edge_attr[:,3]
# flag = data.edge_attr[:,4]
vol0 = data.edge_attr[:,4]
vol1 = data.edge_attr[:,5]
vol10 = data.edge_attr[:,6]

pressure = data.pressure[:,150]
flowrate = data.flowrate[:,150]

edge_index = data.edge_index.numpy()

x = length.numpy()
y = node_to_edge(flowrate.numpy(), edge_index)

print(x.shape, y.shape)

plt.scatter(x, y)
plt.show()

In [ ]:
import sys
sys.path.insert(1, '/data1/tam/python_graph_utilities_v2/Codes/')
from networks_v1 import *

latent_size = 128
hidden_size = 256
n_hidden_per_processor = 2

model = ProcessorLayer(
    node_input_size=latent_size,
    edge_input_size=latent_size,
    node_output_size=latent_size,
    edge_output_size=latent_size,
    hidden_size=hidden_size,
    n_hidden=n_hidden_per_processor
)


In [ ]:
class ShellScriptReader():
    def __init__(self, file_name = None, read_mode = 'r+') -> None:
        self._file = None
        if file_name is not None:
            self.open(file_name, read_mode)
    def open(self, file_name, read_mode):
        if self._file is not None:
            self._file.close()
        self._file = open(file_name, read_mode)
    def close(self):
        if self._file is not None:
            self._file.close()
            self._file = None
    def read(self):
        if self._file is None:
            pass
        file_str = self._file.read()
        lines = file_str.split('\n')

        file_dict = {}
        for i in range(len(lines)):
            lines[i] = lines[i].split('#')[0]
            lines[i] = lines[i].replace(' ','')
                
            line = lines[i].split('=')
            if len(line) == 2:
                file_dict[line[0]] = line[1]
        return file_dict

reader = ShellScriptReader('./test.sh')
d = reader.read()
d['v_TLC']

In [ ]:
import sys
sys.path.insert(1, '/data1/tam/python_graph_utilities_v2/Codes/')
import torch
from dataset import OneDDatasetBuilder, OneDDatasetLoader

# Build dataset
dataset = OneDDatasetBuilder(
    raw_dir='/data1/tam/datasets_231228',
    root_dir='/data1/tam/downloaded_datasets_WT_v2',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)

In [ ]:
dataset[0]

In [ ]:
data = dataset[0]
globalfield = torch.cat([data.global_attr.unsqueeze(0)]*data.node_attr.size(0), dim=0)
globalfield.size()

In [ ]:
import sys
sys.path.insert(1, '/data1/tam/python_graph_utilities_v2/Codes/')
import torch
from dataset import OneDDatasetBuilder, OneDDatasetLoader
from data import TorchGraphData
from torch_geometric.data import DataLoader

data = TorchGraphData(
    x=torch.Tensor([[1,2,3],[4,5,6],[1,2,4],[5,6,7]]),
    edge_index=torch.Tensor([[0,1,1],[1,2,3]])
)
print(data)
print(data.x, data.edge_index)
loader = DataLoader([data]*10, batch_size=3)
data1 = next(iter(loader))
print(data1)
print(data1.x, data1.edge_index)

In [5]:
import sys
sys.path.insert(1, '/data1/tam/python_graph_utilities_v2/Codes/')
import torch
from networks_v3 import PARC_Graph

model = PARC_Graph(
    n_field=2,
    n_meshfield=(3,8),
    n_boundaryfield=0,
    hidden_size=128,
    unet_depth=3,
)

edge_index = torch.Tensor([[0,1,1,2],[1,2,3,4]]).type(torch.LongTensor)
x = torch.rand(size=(5, 2)).float()
node_attr = torch.rand(size=(5, 3)).float()
edge_attr = torch.rand(size=(4, 8)).float()

out, out_dot = model(F_input=x, meshfield=(node_attr, edge_attr), edge_index=edge_index, n_time=10)
out

tensor([[[ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00]],

        [[-8.7996e-05, -1.2583e-04],
         [-5.5656e-05, -3.8496e-05],
         [-7.4919e-05,  3.7570e-05],
         [-1.6839e-05, -7.1785e-05],
         [-2.9094e-05, -1.6161e-04],
         [-1.0411e-04, -6.5836e-05],
         [-1.0761e-05, -8.4182e-05],
         [ 4.2828e-05, -1.1201e-04],
         [-1.9316e-05, -1.4307e-04]],

        [[-8.1565e-04,  1.5254e-05],
         [-1.3695e-04, -1.2381e-05],
         [-8.4752e-04, -7.0432e-04],
         [-2.2037e-04, -1.9480e-04],
         [-6.9423e-04, -8.2848e-05],
         [-7.0981e-05, -3.1515e-04],
         [ 1.8779e-04, -4.6645e-05],
         [-8.7223e-05,  2.8770e-04],
         [-1.0223e-04, -2.1118e-04

In [30]:
import sys
sys.path.insert(1, '/data1/tam/python_graph_utilities_v2/Codes/')
from dataset import OneDDatasetBuilder, OneDDatasetLoader
import numpy as np
from preprocessing import get_graph_partition
from data import TorchGraphData

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_WT_v1',
    sub_dir='processed',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)

data = dataset[0]

def cut_branch(data: TorchGraphData, max_gen: int):
    
    gen = data.edge_attr.numpy()[:,2]
    remain_edge = np.where(gen <= max_gen)[0] # branch id after cutting
    remain_node = data.edge_index.numpy()[:,remain_edge]
    remain_node = np.unique(np.concatenate([remain_node[0], remain_node[1]], axis=0))
    
    return get_graph_partition(data, remain_node, recursive=False)

data = cut_branch(data, max_gen=4)
data

TorchGraphData(edge_index=[2, 34], edge_attr=[34, 15], node_attr=[35, 3], pressure=[35, 201], flowrate=[35, 201], pressure_dot=[59769, 201], flowrate_dot=[59769, 201])